## Instruções
- Redes Neuronais + Backpropagation

Neste trabalho você irá implementar uma rede neuronal com três camadas:

1. Camada de entrada: cada unidade representa uma dimensão do dado de entrada.

2. Camada oculta: cada unidade representa uma transformação a partir das unidades de entrada.

3. Camada de saída: cada unidade representa a chance da saída correspondente ser a correta.

Você irá utilizar a função Sigmóide para obter não-linearidade. Além disso, a função de perda a ser minimizada é a seguinte:

![title](formula.jpg)

onde m é a quantidade de entradas no treino, K é o número de saídas possíveis,  representa a saída correta de cada classe k em cada entrada (i), e similarmente representa a saída dada pela rede neuronal.

O dado a ser utilizado está anexado. Trata-se de 5000 entradas, onde cada entrada refere-se a um dígito escrito manualmente (i.e., MNIST dataset). Dessa forma, m=5000 e K=10. Cada entrada é dada por uma matriz de dimensões 28 por 28, ou seja, um vetor de 784 dimensões. A primeira coluna do arquivo sempre é o rótulo do dígito correto.

A rede neuronal a ser implementada deverá ter 784 unidades de entrada e 10 unidades de saída. Em seus experimentos, você deverá variar o número de unidades na camada oculta (25, 50, 100).

Além disso, você deverá comparar os seguintes algoritmos de cálculo de gradiente:

1. Gradient Descent: o gradiente é calculado após cada época (após as 5000 entradas serem processadas).

2. Stochastic Gradient Descent: o gradiente é calculado após cada entrada.

3. Mini-Batch: o gradiente é calculado após um certo número de entradas (considere 10 e 50).

Por fim, você também deverá variar a taxa de aprendizado: 0.5, 1, 10.

O documento a ser entregue deverá apresentar o resultado de seus experimentos. Ou seja, deverá apresentar discussão da variação do número de unidades na camada oculta para cada um dos três algoritmos de cálculo de gradiente. Você deverá apresentar gráficos mostrando a convergência do erro empírico para cada situação (unidades na camada oculta, algoritmo de cálculo do gradiente, taxa de aprendizado). Você deverá deixar claras todas as hipóteses que julgar serem pertinentes.

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras import optimizers
import numpy

# fix random seed for reproducibility
numpy.random.seed(7)

/home/lfmendes/data/mestrado/machine-learning/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Carregando Arquivo
Carregando o arquivo com os 5000 digitos e fazendo uma divisão de folds para fazer o cross-fold

In [15]:
# load pima indians dataset
dataset = numpy.loadtxt("/home/lfmendes/data/mestrado/machine-learning/ml-neural/data_tp1.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,1:]
y = dataset[:,0]

y_cat = np_utils.to_categorical(y)
num_classes = y_cat.shape[1]

num_pixels = 78

print("Número de entradas: %s" % (len(Y)))
print("Número de features: %s" % (len(X[0])))
print("Número de classes: %s" % (num_classes))

NameError: name 'Y' is not defined

Criando um divisão de 5 folds stratified, isto é, mantendo o número de classes por fold

In [16]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, random_state=42)

## Código para criação do modelo dado os parâmetros que podem ser variados

In [17]:
def create_model(learning_rate=0.5,neuronios_ocultos=25,decay=1e-6,momentum=0.9):
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(neuronios_ocultos, activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    
    # Compile model
    sgd = optimizers.SGD(lr=learning_rate, decay=decay, momentum=momentum)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    return model

## Definindo parâmetros de teste

In [18]:
batch_sizes=[1,10,50,5000]
learning_rates=[0.5, 1, 10]
neuronios=[25,50,100]

# Dividindo em treino e teste

In [19]:
fold = 0
avg_error = 0.0
avg_acc = 0.0

batch_size = 100
learning_rate=0.5
epochs=10
neuronio=25

for train_index, test_index in skf.split(X, y):
    print("Fazendo teste nos fold " + str(fold))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    
    model=create_model(learning_rate=learning_rate,neuronios_ocultos=neuronio,decay=1e-6,momentum=0.9)
    
    # Fit the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=0)
   
    # Final evaluation of the model
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Baseline Error: %.2f%%" % (100-scores[1]*100))
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    avg_acc = avg_acc + model.metrics_names[1]
    
    fold=fold+1
    
print(avg_acc/5)    

Fazendo teste nos fold 0


ValueError: Error when checking input: expected dense_1_input to have shape (78,) but got array with shape (784,)